In [ ]:
# ollama create example -f Modelfile

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pickle
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")
embeddings = HuggingFaceEmbeddings(model_name="avsolatorio/GIST-Embedding-v0")

In [7]:
# url = "https://docs.streamlit.io/"
# loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)
# docs = loader.load()
# len(docs)
# for i in docs:
#     if "content_type" in i.metadata:
#         del i.metadata["content_type"]
#     if "language" in i.metadata:
#         del i.metadata["language"]
#     if "description" in i.metadata:
# del i.metadata["description"]
# pickle.dump(docs, open("docs", "wb"))

docs = pickle.load(open("docs", 'rb'))

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
# db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


In [8]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional

generator = TestsetGenerator.from_langchain(llm, llm, embeddings)
testset = generator.generate_with_langchain_docs(docs, test_size=100, distributions={
                                                 simple: 0.25, reasoning: 0.25, multi_context: 0.25, conditional: 0.25})

embedding nodes:   0%|          | 0/646 [00:00<?, ?it/s]

c:\Users\Raunaq\Desktop\rag_eval\.rag\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
